### Average grid cells, and save each day to a JSON

In [ ]:
import netCDF4 as nc
import numpy as np

from alive_progress import alive_it

from pathlib import Path
from collections import defaultdict

data_dir = Path(r"C:\Users\mathe\Documents\Globus Downloads")

In [ ]:

var_names = {"sea-ice": "siconc", "atmos-near-surface-air-temp": "tas"}

vars = defaultdict(lambda: dict())
total_dirs = len([x for x in data_dir.iterdir() if x.is_dir()])
for var_dir in data_dir.iterdir():
    if not var_dir.is_dir():
        continue
    
    var_name = var_dir.name
    total_files = len(list(var_dir.iterdir()))

    file_bar = alive_it(var_dir.iterdir(), force_tty=True, total=total_files)
    for var_file in file_bar:
        ds = nc.Dataset(var_file)
        days = [int(x) for x in ds.variables["time"]]
        means = [np.mean(x) for x in ds.variables[var_names[var_name]]]
        for i, day in enumerate(days):
            vars[day][var_name] = means[i]



In [ ]:
import json

class Encoder(json.JSONEncoder):
    def default(self, obj):
        if type(obj) == np.float32:
            return int(obj)
        # Let the base class default method raise the TypeError
        return json.JSONEncoder.default(self, obj)


with open("vars.json", "w") as f:
    json.dump(vars, f, cls=Encoder)

In [ ]:
# Convert json to CSV
import csv

with open("vars.json") as in_file:
    json_file = json.load(in_file)
    with open("vars.csv", "w") as out_file:
        csv_file = csv.writer(out_file)
        days = json_file.keys()
        vars = set()
        for day in days:
            vars |= set(json_file[day].keys())
        csv_file.writerow(("Day", *vars))
        for day in days:
            row = [str(day), "", ""]
            for i, var in enumerate(vars):
                if var in json_file[day]:
                    row[i+1] = json_file[day][var]
            csv_file.writerow(row)

# Plot all points of a file on a map to guage the extent of the area covered

In [ ]:
FILE_NUM = 0

records = defaultdict(lambda: dict())
var_dir = data_dir / 'atmos-near-surface-air-temp'

vars = nc.Dataset(list(var_dir.iterdir())[FILE_NUM]).variables
days = [int(x) for x in vars["time"][0:10]]
means = [np.mean(x) for x in vars["tas"][0:10]]
for i, day in enumerate(days):
    records[day]["atmos-near-surface-air-temp"] = means[i]

var_dir = data_dir / 'sea-ice'
vars = nc.Dataset(list(var_dir.iterdir())[FILE_NUM]).variables
days = [int(x) for x in vars["time"][0:10]]
means = [np.mean(x) for x in vars["siconc"][0:10]]
for i, day in enumerate(days):
    records[day]["sea-ice"] = means[day]

print(records.keys())



In [ ]:
vars["time"][0:10]

In [ ]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

from itertools import product

FILE_NUM = 0

records = defaultdict(lambda: dict())
var_dir = data_dir / 'atmos-near-surface-air-temp'

points = [Point(xy) for xy in zip(vars["longitude"], vars["latitude"])]
gdf = GeoDataFrame(None, geometry=points)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=1);